# **Cassava competition Submission**

이 submission 노트북은 https://www.kaggle.com/vkehfdl1/for-korean-lb-0-895-effnetb4-train 이 train 노트북에서 이어지는 노트북입니다! 보고 오시면 큰 도움이 됩니다. 

이 노트북에서 앞선 노트북에서 훈련한 모델을 ensemble하고 TTA 이후 submission하는 과정을 살펴보겠습니다. 

여기서 사용하는 데이터셋은 직접 훈련한 모델들을 모아놓은 데이터셋입니다. https://www.kaggle.com/vkehfdl1/cassava-my-models 

In [ ]:
import numpy as np 
import pandas as pd 
from PIL import Image
import os
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import RandomCrop,CenterCrop, RandomRotation
import tensorflow_addons as tfa


from sklearn.utils import shuffle

In [ ]:
fold = 4 #train 단계에서의 fold 개수를 지정. 앞선 노트북에서 4 fold를 사용했으므로 4로 적어줌

In [ ]:
models = list() #각 모델을 넣어줄 리스트를 만듬
for i in range (fold):
    models.append(tf.keras.models.load_model('../input/cassava-my-models/effnetb4_2019_4fold_onlythebest/'+str(i) + 'fold.h5')) #각 fold를 훈련한 4개의 모델을 입력

In [ ]:
def scan_over_image(img_path, crop_size=500): #이미지를 로드하는 함수. train과 마찬가지로 500x500 사이즈의 이미지를 로드
    
    img = Image.open(img_path) #pillow를 이용하여 주어진 경로의 이미지를 열어줌
    img_height, img_width = img.size #이미지의 사이즈를 받아 저장
    img = np.array(img) #이미지를 numpy array로 변환
    
    """
    아래의 코드는 이미지를 총 4개로 자르는 코드입니다. 
    500x500 사이즈로 이미지를 잘라주는데, 이 때 겹치는 부분이 최대한 없게 끝부터 잘라줍니다.
    test 이미지는 600x800 이미지입니다. 왼쪽 위부터 (0,0) 이라고 하면 잘리는 이미지는 다음과 같습니다.
    1. (0~499, 0~499)
    2. (100~599, 0~499)
    3. (0~499, 300~799)
    4. (100~599, 300~799)
    """
    y = random.randint(0,img_height-crop_size) 
    x = random.randint(0,img_width-crop_size)

    x_img_origins = [0,img_width-crop_size]
    y_img_origins = [0,img_height-crop_size]
    img_list = []
    for x in x_img_origins:
        for y in y_img_origins:
            img_list.append(img[x:x+crop_size , y:y+crop_size,:])
  
    return np.array(img_list) #생성한 이미지 4장을 반환합니다.

여기서는 TTA라는 기술을 사용합니다. Test Time Augmentation의 약자로써, test 이미지를 augmentation을 통하여 여러 개로 생성하고, 각 이미지를 각각 예측하여 가장 확률이 높은 class를 최종 결과로 예측합니다. 
여기서는 4장의 이미지를 4개의 모델에 돌려 한 이미지를 총 16번 예측하게 됩니다.

In [ ]:
test_time_augmentation_layers = tf.keras.Sequential( #이미지를 augmentation 할 수 있는 layer를 선언
    [
        layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"), #랜덤으로 이미지를 좌우로 뒤집어줌
        layers.experimental.preprocessing.RandomRotation(0.25), #이미지를 좌우로 25% 이내로 랜덤으로 돌림
        layers.experimental.preprocessing.RandomZoom((-0.2, 0)) #이미지를 0~20%만큼 랜덤으로 축소
    ]
)

In [ ]:
def predict_and_vote(image_filename, folder, fold=4, models=models): #TTA 및 ensemble을 위한 함수
    localised_predictions = [] #prediction 값을 담을 리스트 정의
    local_image_list = scan_over_image(folder+image_filename) #이미지 4장을 리스트에 담음
    models = shuffle(models) #한 모델이 한 부분의 사진만 계속 예측하는것을 막기 위해 모델을 랜덤으로 섞음
    
    for local_image in (local_image_list): 
        duplicated_local_image = tf.convert_to_tensor(np.array([local_image for i in range(fold)])) #각 이미지를 텐서로 변환
        augmented_images = test_time_augmentation_layers(duplicated_local_image) #텐서로 변환한 이미지를 augmentation을 통해 살짝 변형을 해줌
        for i in range (fold): 
            predictions = models[i].predict(augmented_images) #변형한 이미지를 각 모델에서 한 번 씩 예측
            localised_predictions.append(np.sum(predictions, axis=0)) #예측값을 리스트에 입력
    
    global_predictions = np.sum(np.array(localised_predictions),axis=0) #각 예측값들을 모두 더해줌
    final_prediction = np.argmax(global_predictions) #모두 더한 예측 값중 가장 큰 값을 최종 결과로 선정
    
    return final_prediction #최종으로 예측한 결과를 반환

In [ ]:
def run_predictions_over_image_list(image_list, folder): #최종 예측을 위해 필요한 함수 선언
    predictions = [] #각 사진의 최종 예측값을 담는 리스트
    with tqdm(total=len(image_list)) as pbar: #예측 진행 상황을 알려주는 그래프 표시를 위한 코드
        for image_filename in image_list:
            pbar.update(1) #예측 진행 상황을 알려주는 그래프 표시를 위한 코드
            predictions.append(predict_and_vote(image_filename, folder)) #각각 사진들을 예측 후 최종 예측값을 리스트에 입력
    return predictions #모든 사진들의 예측값을 반환

In [ ]:
test_folder = '../input/cassava-leaf-disease-classification/test_images/' #test 이미지들이 있는 폴더 경로 지정
submission_df = pd.DataFrame(columns={"image_id","label"}) #submission을 위한 DataFrame 정의
submission_df["image_id"] =  os.listdir(test_folder) #폴더 안의 이미지들의 경로를 입력
submission_df["label"] = 0 #최종 결과 값이 담길 label 정의

In [ ]:
submission_df["label"] = run_predictions_over_image_list(submission_df["image_id"], test_folder) #최종 결과값 계산

In [ ]:
submission_df

In [ ]:
submission_df.to_csv("submission.csv", index=False) #최종 결과값을 제출을 위해 csv 파일로 변환

**도움이 되었다면 업보트 upvote 한 번 씩 부탁드립니다! 감사합니다!**